### Notebook con fragmentos de códigos empleados para probar conceptos antes de aplicarlos al código final. No tener en cuenta

In [1]:
import os
import sys
import pandas as pd
import open3d as o3d

sys.path.append(r'D:\Nicolas\Posgrado\Trabajos y Tesis\LIDAR\LIDAR_super_resolution\Scripts\otras_arquitecturas\3_pytorch_interpolation')

#import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torchsummary import summary
import time
#import kaolin
#from data_gen_distance import *
from pointcloud_utils_functions_v2 import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [18]:
x = torch.tensor([[[1.0,2.0,3.0,4.0,5.0], [5.0,6.0,7.0,8.0,9.0], [9.0,8.0,7.0,6.0,5.0], [5.0,4.0,3.0,2.0,1.0], [1.0,1.0,1.0,1.0,1.0]]]).unsqueeze(0)

hr = torch.ones((1, 1, 10, 5))
row_pos, column_pos = torch.where(hr[0,0] >= 0.0)
odd_row_pos = row_pos[row_pos % 2 != 0] #/ row_pos.max()
column_pos = column_pos[:column_pos.shape[0]//2] #/ column_pos.max()
new_pixel_coords = torch.Tensor(list(zip(odd_row_pos, column_pos))[:-5])

def get_windows(x, new_pixel_coords, upscaling_factor=2):
    #def __init__(self, upsampling_factor=2, n_channels=1, padding=(0,1), stride=(1,1)): 
    #padding:(agregar n filas, agregar n columnas)
    #stride(de a cuantas filas me muevo:de a cuantas columnas me muevo)
    kernel_size = (4,3)
    padding = (1,1)
    stride = (1,1)
    new_pixel_coords_batch = new_pixel_coords.unsqueeze(0).repeat(x.shape[0],1,1)

    if upscaling_factor == 2:
        windows = F.unfold(x, kernel_size=kernel_size, padding=padding, stride=stride)
        windows = windows.transpose(1, 2) #Obtener los valores de la ventana o kernel ordenados por fila, donde cada fila representa una ventana serializada
        windows = torch.dstack((windows, new_pixel_coords_batch))
    else:
        print("ERROR: Wrong Upsampling factor")
    return windows

windows = get_windows(x, new_pixel_coords)
print(windows.shape)
print(x.shape)



torch.Size([1, 20, 14])
torch.Size([1, 1, 5, 5])


In [10]:
p1 = torch.tensor([61.4734, 19.1356, -2.4600])
print(p1.shape)
torch.linalg.norm(p1, 2, axis=0)

torch.Size([3])


tensor(64.4298)

In [12]:
import numpy as np
import kaolin

p1 = np.array([[[8.8977, 4.1709, 1.2839]]])
p2 = np.array([[[6.9340, 6.1152, 3.4435]]])

def chamfer_distance(points1, points2):
    # Calcular las distancias desde cada punto en points1 al conjunto points2
    distances1to2 = np.min(np.linalg.norm(points1[:, np.newaxis] - points2, axis=-1), axis=1)
    
    # Calcular las distancias desde cada punto en points2 al conjunto points1
    distances2to1 = np.min(np.linalg.norm(points2[:, np.newaxis] - points1, axis=-1), axis=1)
    
    # Sumar las distancias mínimas en ambas direcciones para obtener la distancia de Chamfer
    chamfer_dist = np.sum(distances1to2) + np.sum(distances2to1)
    
    return chamfer_dist

print(chamfer_distance(p1[0],p2[0]))

p1 = torch.tensor([[[8.8977, 4.1709, 1.2839]]])
p2 = torch.tensor([[[6.9340, 6.1152, 3.4435]]])
dist = kaolin.metrics.pointcloud.chamfer_distance(p1.to('cuda'), p2.to('cuda'), squared=True)
print(dist)

7.0143545219784835
tensor([24.6006], device='cuda:0')


In [2]:
dataset_path = r'D:\Nicolas\Posgrado\Trabajos y Tesis\LIDAR\Datasets LIDAR\kitti\kitti_3d_object\training'
pointcloud_file = '000034'
pointcloud_fullpath = os.path.join(dataset_path, 'velodyne', rf'{pointcloud_file}.bin')
labels_path = os.path.join(dataset_path, 'label_2', rf'{pointcloud_file}.txt')

pointcloud = read_bin(pointcloud_fullpath)

label_data = pd.read_csv(labels_path, sep=' ', header=None, names=['type', 'truncated', 'occluded', 'alpha', 'bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom',
                                                                     'height_object', 'width_object', 'length_object', 'location_x_camera', 'location_y_camera', 
                                                                     'location_z_camera', 'rotation_y'])
label_data = label_data[label_data['type']!='DontCare']
label_data

,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,height_object,width_object,length_object,location_x_camera,location_y_camera,location_z_camera,rotation_y
0,Car,0.0,0,0.50,46.17,196.15,328.40,286.09,1.56,1.71,4.5,-8.15,2.06,14.12,-0.02
1,Car,0.0,1,-2.99,466.96,184.85,522.12,207.02,1.44,1.52,3.5,-7.84,2.28,49.04,3.13


In [3]:
pd_pointcloud = pd.DataFrame(pointcloud, columns=['X', 'Y', 'Z', 'I'])
pd_pointcloud

,X,Y,Z,I
0,60.226002,6.967,2.259,0.00
1,60.341000,7.173,2.264,0.00
2,60.366001,7.368,2.266,0.03
3,61.824001,7.744,2.316,0.09
4,61.334999,7.781,2.300,0.08
...,...,...,...,...
121888,3.713000,-1.424,-1.738,0.28
121889,3.722000,-1.415,-1.741,0.32
121890,3.735000,-1.406,-1.745,0.30
121891,3.621000,-1.356,-1.687,0.00


In [4]:
boundary = {
    "minX": 0,
    "maxX": 50,
    "minY": -25,
    "maxY": 25,
    "minZ": -2.73,
    "maxZ": 1.27
}

def removePoints(PointCloud, BoundaryCond):
    # Boundary condition
    minX = BoundaryCond['minX']
    maxX = BoundaryCond['maxX']
    minY = BoundaryCond['minY']
    maxY = BoundaryCond['maxY']
    minZ = BoundaryCond['minZ']
    maxZ = BoundaryCond['maxZ']

    # Remove the point out of range x,y,z
    mask = np.where((PointCloud[:, 0] >= minX) & (PointCloud[:, 0] <= maxX) & (PointCloud[:, 1] >= minY) & (
            PointCloud[:, 1] <= maxY) & (PointCloud[:, 2] >= minZ) & (PointCloud[:, 2] <= maxZ))
    PointCloud = PointCloud[mask]

    return PointCloud

pointcloud = removePoints(pointcloud, boundary)

In [5]:
#source: https://github.com/HengLan/Visualize-KITTI-Objects-in-Videos/blob/main/utility.py#L45
# https://github.com/HengLan/Visualize-KITTI-Objects-in-Videos/blob/main/KITTI.py#L269
def transform_3dbox_to_pointcloud(dimension, location, rotation):
    """
    convert the 3d box to coordinates in pointcloud
    :param dimension: height, width, and length
    :param location: x, y, and z
    :param rotation: rotation parameter
    :return: transformed coordinates
    """
    height, width, lenght = dimension
    x_offset = 0#lenght/2 #/ 2
    y_offset = 0.2#height / 4
    z_offset = 0#width/2 #/ 2

    x, y, z = location
    x_corners = [lenght/2 + x_offset, lenght/2 + x_offset, -lenght/2 - x_offset, -lenght/2 - x_offset,  lenght/2 + x_offset,  lenght/2 + x_offset, -lenght/2 - x_offset, -lenght/2 - x_offset]
    y_corners = [y_offset, y_offset, y_offset, y_offset, -height - y_offset, -height - y_offset, -height - y_offset, -height - y_offset]
    z_corners = [width/2 + z_offset, -width/2 - z_offset, -width/2 - z_offset, width/2 + z_offset, width/2 + z_offset, -width/2 - z_offset, -width/2 - z_offset, width/2 + z_offset]

    corners_3d = np.vstack([x_corners, y_corners, z_corners])

    # transform 3d box based on rotation along Y-axis
    R_matrix = np.array([[np.cos(rotation), 0, np.sin(rotation)],
                         [0, 1, 0],
                         [-np.sin(rotation), 0, np.cos(rotation)]])

    corners_3d = np.dot(R_matrix, corners_3d).T

    # shift the corners to from origin to location
    corners_3d = corners_3d + np.array([x, y, z])

    # from camera coordinate to velodyne coordinate
    corners_3d = corners_3d[:, [2, 0, 1]] * np.array([[1, -1, -1]])

    return corners_3d

def get_3d_corners(label_data):
    ''' x-axis points to the front
        y-axis points to left
        corners: (8,3) array of vertices for the 3d box in following order [X, Y, Z]:
            6 -------- 7
           /|         /|
          5 -------- 4 .
          | |        | |
          . 0 -------- 1
          |/         |/
          3 -------- 2
    '''
    corners_3d = []
    for i in range(len(label_data.axes[0])):
        dimension = [label_data.iloc[i].height_object, label_data.iloc[i].width_object, label_data.iloc[i].length_object]
        location = [label_data.iloc[i].location_x_camera, label_data.iloc[i].location_y_camera, label_data.iloc[i].location_z_camera]
        rotation = label_data.iloc[i].rotation_y

        corners_3d.append(transform_3dbox_to_pointcloud(dimension, location, rotation))                                                                                          
        #corners_3d.append([transform_3dbox_to_pointcloud(dimension, location, rotation), rotation])
    return corners_3d

corners_all = get_3d_corners(label_data)
#print(corners_all[0])
#print(len(corners_all[0]))
#print(len(corners_all))

#for idx, corners in enumerate(corners_all):
#    print(idx, corners[1])

In [6]:
#Lento, pero SI extrae exactamente los puntos dentro del bbox aunque esten desalineados con el centro del mundo
'''
If your cube vertices are not aligned with the world coordinate system (i.e., they are in arbitrary orientations), you will need to use a more complex approach
to determine if a point is inside the cube. One common method is to use the concept of convex hulls. Here's how you can approach this problem:

Find the convex hull of the cube: The convex hull is the smallest convex shape that contains all the cube's vertices.
You can use algorithms like the QuickHull algorithm or the Gift Wrapping algorithm to find the convex hull of your cube's vertices.

Check if the point is inside the convex hull: Once you have the convex hull, you can check if the point is inside it. 
You can use the point-in-polygon test to determine this.
'''

from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay

def filter_points(pointcloud, corners):
    filtered_points = pd.DataFrame(columns=['X', 'Y', 'Z', 'I'])

    # Find the convex hull of the cube's vertices
    hull = ConvexHull(corners)

    # Check if the point is inside the convex hull  
    delaunay_triangulation = Delaunay(hull.points[hull.vertices])
    for i, point in enumerate(pointcloud):
        simplex = delaunay_triangulation.find_simplex(point[:-1])
        if simplex >= 0:
            filtered_points.loc[i] = point
    return filtered_points

for idx, corners in enumerate(corners_all):
    filtered_points = filter_points(pointcloud, corners)

    #Add 3d box vertices to visualize it
    corners_pd = pd.DataFrame(corners, columns = ['X','Y','Z'])
    corners_pd['I'] = 0.0
    
    #point=0
    if idx == 0:
        filtered_pointcloud = filtered_points.copy()
        filtered_pointcloud_with_bbox = filtered_points.copy()
        #filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd.iloc[[point]]], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)
    else:
        filtered_pointcloud = pd.concat([filtered_pointcloud, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)

filtered_pointcloud.reset_index(inplace=True)
filtered_pointcloud.drop(['index'], axis=1, inplace=True)

print(filtered_pointcloud)

             X      Y      Z     I
0    14.911000  8.194 -0.340  0.27
1    14.904000  8.251 -0.341  0.33
2    14.923000  8.292 -0.342  0.38
3    14.923000  8.354 -0.343  0.37
4    14.924000  8.416 -0.344  0.34
..         ...    ...    ...   ...
790  48.724998  8.445 -1.378  0.00
791  48.747002  8.606 -1.380  0.00
792  48.519001  8.881 -1.374  0.16
793  48.360001  8.930 -1.370  0.00
794  48.459000  9.106 -1.374  0.00

[795 rows x 4 columns]


In [80]:
#Otra opción pero no da bien el resultado
def is_point_inside_cube(cube_vertices, inv_rotation_matrix, point):
    
    # Transform the cube vertices
    transformed_cube_vertices = np.dot(inv_rotation_matrix, cube_vertices.T).T
    
    # Transform the point
    transformed_point = np.dot(inv_rotation_matrix, point)

    # Calculate the boundaries of the transformed cube
    min_x = np.min(transformed_cube_vertices[:, 0])
    max_x = np.max(transformed_cube_vertices[:, 0])
    min_y = np.min(transformed_cube_vertices[:, 1])
    max_y = np.max(transformed_cube_vertices[:, 1])
    min_z = np.min(transformed_cube_vertices[:, 2])
    max_z = np.max(transformed_cube_vertices[:, 2])

    # Check if the transformed point is inside the transformed cube
    return (
        min_x <= transformed_point[0] <= max_x and
        min_y <= transformed_point[1] <= max_y and
        min_z <= transformed_point[2] <= max_z
    )

def filter_points(pointcloud, corners, inv_rotation_matrix):
    filtered_points = pd.DataFrame(columns=['X', 'Y', 'Z', 'I'])
    for i, point in enumerate(pointcloud):       
        if is_point_inside_cube(corners, inv_rotation_matrix, point[:-1]): #Point must be [X,Y,Z] without I
            filtered_points.loc[i] = point
    return filtered_points

for idx, corners in enumerate(corners_all):
    R_matrix = np.array([[np.cos(corners[1]), 0, np.sin(corners[1])],
                         [0, 1, 0],
                         [-np.sin(corners[1]), 0, np.cos(corners[1])]])

    # Inverse of the rotation matrix
    inv_rotation_matrix = np.linalg.inv(R_matrix)

    filtered_points = filter_points(pointcloud, corners[0], inv_rotation_matrix)

    #Add 3d box vertices to visualize it
    corners_pd = pd.DataFrame(corners[0], columns = ['X','Y','Z'])
    corners_pd['I'] = 0.0
    
    #point=0
    if idx == 0:
        filtered_pointcloud = filtered_points.copy()
        filtered_pointcloud_with_bbox = filtered_points.copy()
        #filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd.iloc[[point]]], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)
    else:
        filtered_pointcloud = pd.concat([filtered_pointcloud, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)

filtered_pointcloud.reset_index(inplace=True)
filtered_pointcloud.drop(['index'], axis=1, inplace=True)

print(filtered_pointcloud)


               X      Y      Z     I
0       3.607000  2.640 -0.105  0.00
1       3.577000  2.661 -0.105  0.00
2       3.542000  2.705 -0.104  0.00
3       3.524000  2.727 -0.104  0.00
4       3.490000  2.709 -0.102  0.00
...          ...    ...    ...   ...
22088  18.143000 -6.773 -1.702  0.29
22089  17.337000 -7.523 -1.772  0.32
22090  17.372999 -7.474 -1.774  0.36
22091  17.337999 -7.394 -1.767  0.32
22092  17.295000 -7.312 -1.760  0.33

[22093 rows x 4 columns]


In [8]:
#Eficiente, pero no extrae exactamente los puntos dentro del bbox porque los bbox no estan alineados con el centro del mundo

def filter_points(pd_pointcloud, corners):
    print(pd_pointcloud.shape)
    print(corners.shape)
    filtered_points = pd_pointcloud[(corners[:,0].min() < pd_pointcloud['X']) & (pd_pointcloud['X'] < corners[:,0].max())
                                      & (corners[:,1].min() < pd_pointcloud['Y']) & (pd_pointcloud['Y'] < corners[:,1].max())
                                      & (corners[:,2].min() < pd_pointcloud['Z']) & (pd_pointcloud['Z'] < corners[:,2].max())]
    print(type(filtered_points))
    return filtered_points

for idx, corners in enumerate(corners_all):
    filtered_points = filter_points(pd_pointcloud, corners)
    
    #Add 3d box vertices to visualize it
    corners_pd = pd.DataFrame(corners, columns = ['X','Y','Z'])
    corners_pd['I'] = 0.0
    
    #point=0
    if idx == 0:
        filtered_pointcloud = filtered_points.copy()
        filtered_pointcloud_with_bbox = filtered_points.copy()
        #filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd.iloc[[point]]], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)
    else:
        filtered_pointcloud = pd.concat([filtered_pointcloud, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, filtered_points], ignore_index=True)
        filtered_pointcloud_with_bbox = pd.concat([filtered_pointcloud_with_bbox, corners_pd], ignore_index=True)

filtered_pointcloud.reset_index(inplace=True)
filtered_pointcloud.drop(['index'], axis=1, inplace=True)

print(filtered_pointcloud)

(121893, 4)
(8, 3)
<class 'pandas.core.frame.DataFrame'>
(121893, 4)
(8, 3)
<class 'pandas.core.frame.DataFrame'>
             X      Y      Z     I
0    14.911000  8.194 -0.340  0.27
1    14.904000  8.251 -0.341  0.33
2    14.923000  8.292 -0.342  0.38
3    14.923000  8.354 -0.343  0.37
4    14.924000  8.416 -0.344  0.34
..         ...    ...    ...   ...
803  48.724998  8.445 -1.378  0.00
804  48.747002  8.606 -1.380  0.00
805  48.519001  8.881 -1.374  0.16
806  48.360001  8.930 -1.370  0.00
807  48.459000  9.106 -1.374  0.00

[808 rows x 4 columns]


In [6]:
def get_outer_points(pd_pointcloud, filtered_pointcloud):

    # Crear los DataFrames de ejemplo
    df1 = pd_pointcloud.copy()
    df2 = filtered_pointcloud.copy()

    # Realizar un merge con indicator=True
    merged = df1.merge(df2, how='outer', indicator=True)

    # Filtrar las filas que están en ambos DataFrames
    result = merged[merged['_merge'] == 'both']
    # Eliminar las filas repetidas de df1
    df1 = df1[~df1.isin(result.to_dict('list')).all(1)]

    # Eliminar las filas repetidas de df2 (opcional)
    #df2 = df2[~df2.isin(result.to_dict('list')).all(1)]

    return df1

a = get_outer_points(pd_pointcloud, filtered_pointcloud)
#print(pd_pointcloud.shape)
#print(filtered_pointcloud.shape)
#print(a.shape)

(121893, 4)
(808, 4)
(121085, 4)


In [12]:
filtered_pointcloud

,X,Y,Z,I
0,14.911000,8.194,-0.340,0.27
1,14.904000,8.251,-0.341,0.33
2,14.923000,8.292,-0.342,0.38
3,14.923000,8.354,-0.343,0.37
4,14.924000,8.416,-0.344,0.34
...,...,...,...,...
803,48.724998,8.445,-1.378,0.00
804,48.747002,8.606,-1.380,0.00
805,48.519001,8.881,-1.374,0.16
806,48.360001,8.930,-1.370,0.00


In [9]:
save_path = r'D:\Nicolas\reducida.ply'
save_ply(filtered_pointcloud.to_numpy(), save_path)

save_path = r'D:\Nicolas\reducida_with_bbox.ply'
save_ply(filtered_pointcloud_with_bbox.to_numpy(), save_path)

save_path_or = r'D:\Nicolas\original.ply'
save_ply(pointcloud, save_path_or)

Pytorch Loss Module Test

In [2]:
import sys
sys.path.append(r'D:\Nicolas\Posgrado\Trabajos y Tesis\LIDAR\LIDAR_super_resolution\Scripts\otras_arquitecturas\3_pytorch_interpolation')

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import kaolin
from object_filtering import pointcloud_filter
from pointcloud_utils_functions_v2 import *


In [38]:
class ChamferLoss(nn.Module):
    def __init__(self, device):
        super(ChamferLoss, self).__init__()
        self.device = device

    def forward(self, image_pred, image_gt, label_path):
        """
        Compute the Chamfer distance between predicted and ground truth point clouds.

        Args:
            point_cloud_pred (torch.Tensor): Predicted point cloud tensor of shape (batch_size, num_points, num_dims).
            point_cloud_gt (torch.Tensor): Ground truth point cloud tensor of shape (batch_size, num_points, num_dims).
        Returns:
            torch.Tensor: Chamfer distance loss.
        """
        image_pred = image_pred.to(device)
        image_gt = image_gt.to(device)

        pointcloud_pred = range_image_to_pointcloud_pytorch(image_pred, device)
        pointcloud_gt = range_image_to_pointcloud_pytorch(image_gt, device)

        pointcloud_np_pred_filtered, pointcloud_np_pred_non_filtered = pointcloud_filter(pointcloud_pred.cpu().numpy()[0], label_path)
        pointcloud_pred_filtered = torch.from_numpy(pointcloud_np_pred_filtered).unsqueeze(0)#.unsqueeze(0) #[batch, num_points, 4]
        pointcloud_pred_non_filtered = torch.from_numpy(pointcloud_np_pred_non_filtered).unsqueeze(0)#.unsqueeze(0) #[batch, num_points, 4]

        pointcloud_np_gt_filtered, pointcloud_np_get_non_filtered = pointcloud_filter(pointcloud_gt.cpu().numpy()[0], label_path)
        pointcloud_gt_filtered = torch.from_numpy(pointcloud_np_gt_filtered).unsqueeze(0) #[batch, num_points, 4]
        pointcloud_gt_non_filtered = torch.from_numpy(pointcloud_np_get_non_filtered).unsqueeze(0) #[batch, num_points, 4]
        
        chamfer_loss_filtered = kaolin.metrics.pointcloud.chamfer_distance(pointcloud_pred_filtered[:,:,:3].to(self.device), pointcloud_gt_filtered[:,:,:3].to(self.device))
        chamfer_loss_non_filtered = kaolin.metrics.pointcloud.chamfer_distance(pointcloud_pred_non_filtered[:,:,:3].to(self.device), pointcloud_gt_non_filtered[:,:,:3].to(self.device))
        
        chamfer_loss_mean = torch.mean(chamfer_loss_filtered) * 0.7 + torch.mean(chamfer_loss_non_filtered) * 0.3

        return chamfer_loss_mean

device = 'cuda'
loss_fn = ChamferLoss(device=device)

In [95]:
dataset_path = r'D:\Nicolas\Posgrado\Trabajos y Tesis\LIDAR\Datasets LIDAR\kitti\kitti_3d_object\training'
pointcloud_fullpath = dataset_path + r'\velodyne\000034.bin'
labels_path = dataset_path + r'\label_2\000034.txt'
pointcloud = read_bin(pointcloud_fullpath)
range_image = pointcloud_to_range_image(pointcloud, size=(64, 2048))
range_image = torch.from_numpy(range_image).unsqueeze(0) #[batch, channel, height, width]

pointcloud_fullpath_2 = dataset_path + r'\velodyne\000035.bin'
labels_path_2 = dataset_path + r'\label_2\000035.txt'
pointcloud_2 = read_bin(pointcloud_fullpath_2)
range_image_2 = pointcloud_to_range_image(pointcloud_2, size=(64, 2048))
range_image_2 = torch.from_numpy(range_image_2).unsqueeze(0) #[batch, channel, height, width]

range_image_batch = torch.stack((range_image, range_image_2), dim=0) #[batch, channels, height, width] [2,1,64,2048]
############################### HASTA AQUI ESTAMOS EN EL MUNDO DE LAS IMÁGENES#####################################

pointcloud_batch = range_image_to_pointcloud_pytorch(range_image_batch.to(device), device='cuda') #[batch, num_points, 4]

def removePoints(PointCloud):
    # Remove the point out of range x,y,z
    BoundaryCond = {"minX": 0, "maxX": 50, "minY": -25, "maxY": 25, "minZ": -2.73, "maxZ": 1.27}

    masks = (
        (PointCloud[:, :, 0] >= BoundaryCond['minX']) & 
        (PointCloud[:, :, 0] <= BoundaryCond['maxX']) & 
        (PointCloud[:, :, 1] >= BoundaryCond['minY']) & 
        (PointCloud[:, :, 1] <= BoundaryCond['maxY']) & 
        (PointCloud[:, :, 2] >= BoundaryCond['minZ']) & 
        (PointCloud[:, :, 2] <= BoundaryCond['maxZ'])
    )

    # Apply the masks to filter the points in each point cloud
    filtered_point_clouds = [PointCloud[mask] for PointCloud, mask in zip(PointCloud, masks)]

    return filtered_point_clouds

#pointcloud_removed = removePoints(pointcloud_batch)

def transform_3dbox_to_pointcloud(dimension, location, rotation):
    """
    #source: https://github.com/HengLan/Visualize-KITTI-Objects-in-Videos/blob/main/utility.py#L45
    # https://github.com/HengLan/Visualize-KITTI-Objects-in-Videos/blob/main/KITTI.py#L269
    convert the 3d box to coordinates in pointcloud
    :param dimension: height, width, and length
    :param location: x, y, and z
    :param rotation: rotation parameter
    :return: transformed coordinates
    """
    height, width, length = dimension
    x_offset = length/4 #/ 2
    y_offset = 0.2#height / 4
    z_offset = width/4 #/ 2

    x, y, z = location
    x_corners = torch.tensor([length / 2 + x_offset, length / 2 + x_offset, -length / 2 - x_offset, -length / 2 - x_offset,
                              length / 2 + x_offset, length / 2 + x_offset, -length / 2 - x_offset, -length / 2 - x_offset])
    y_corners = torch.tensor([y_offset, y_offset, y_offset, y_offset, -height - y_offset, -height - y_offset, -height - y_offset, -height - y_offset])
    z_corners = torch.tensor([width / 2 + z_offset, -width / 2 - z_offset, -width / 2 - z_offset, width / 2 + z_offset,
                              width / 2 + z_offset, -width / 2 - z_offset, -width / 2 - z_offset, width / 2 + z_offset])

    # Create a tensor of corners_3d
    corners_3d = torch.stack([x_corners, y_corners, z_corners])

    # transform 3d box based on rotation along Y-axis
    rotation_matrix = torch.tensor([[torch.cos(rotation), 0, torch.sin(rotation)],
                                    [0, 1, 0],
                                    [-torch.sin(rotation), 0, torch.cos(rotation)]])

    corners_3d = torch.transpose(torch.mm(rotation_matrix, corners_3d), 0, 1)

    # shift the corners to from origin to location
    corners_3d = corners_3d + torch.tensor([x, y, z])

    # from camera coordinate to velodyne coordinate
    corners_3d = corners_3d[:, [2, 0, 1]] * torch.tensor([[1.0, -1.0, -1.0]])

    return corners_3d


dimension = torch.tensor([1.0, 2.0, 3.0])
location = torch.tensor([4.0, 5.0, 6.0])
rotation = torch.tensor(1.0)

result = transform_3dbox_to_pointcloud(dimension, location, rotation)
print(result)

def get_3d_corners(label_data):
    ''' x-axis points to the front
        y-axis points to left
        corners: (8,3) array of vertices for the 3d box in following order [X, Y, Z]:
            6 -------- 7
           /|         /|
          5 -------- 4 .
          | |        | |
          . 0 -------- 1
          |/         |/
          3 -------- 2
    '''
    corners_3d = []
    for i in range(len(label_data.axes[0])):
        dimension = [label_data.iloc[i].height_object, label_data.iloc[i].width_object, label_data.iloc[i].length_object]
        location = [label_data.iloc[i].location_x_camera, label_data.iloc[i].location_y_camera, label_data.iloc[i].location_z_camera]
        rotation = label_data.iloc[i].rotation_y

        corners_3d.append(transform_3dbox_to_pointcloud(dimension, location, rotation))                                                                                          

    return corners_3d

#save_path = r'D:\Nicolas\a.ply'
#save_ply(pointcloud_removed[1].cpu().numpy(), save_path)


#error = loss_fn(range_image_batch, range_image_batch, labels_path)
#print(error)

tensor([[ 4.9171, -6.4779, -5.2000],
        [ 3.2962, -3.9535, -5.2000],
        [ 7.0829, -1.5221, -5.2000],
        [ 8.7038, -4.0465, -5.2000],
        [ 4.9171, -6.4779, -3.8000],
        [ 3.2962, -3.9535, -3.8000],
        [ 7.0829, -1.5221, -3.8000],
        [ 8.7038, -4.0465, -3.8000]])


In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from data_gen_distance import *
from pointcloud_utils_functions_v2 import *
import time

class IterDataset(torch.utils.data.IterableDataset):
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        return self.generator()

batch_size=2
train_dataset = IterDataset(train_data_generator)
valid_dataset = IterDataset(valid_data_generator)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)

lr, hr, labels_path = next(iter(train_dataloader))
labels = read_labels(labels_path)
